<a href="https://colab.research.google.com/github/felixyuzhenyang/MSSP-6070-YuzhenYang/blob/main/PARTICIPATION_ACTIVITY_Week_9_YuzhenYang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium
!apt-get update
!apt-get install -y chromium-chromedriver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 18.9 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 https://cli.github.com/packages stable InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,168 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRe

In [2]:
##### ===== Part 1 Set up & function definition =====

import os
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import sys

def setup_selenium_in_colab():
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    try:
        # Initialize the Chrome driver
        driver = webdriver.Chrome(options=chrome_options)
        return driver
    except Exception as e:
        print(f"Error: could not start WebDriver.")
        print(f"Details: {e}")
        return None

def download_imgur_images(search_term, max_images=10):

    save_directory = f"imgur_{search_term.replace(' ', '_')}"
    os.makedirs(save_directory, exist_ok=True)
    print(f"Images will be saved to: {os.path.abspath(save_directory)}")
    print(f"(You can find it in the 'Files' tab on the left sidebar)")

    search_url = f"https://imgur.com/search?q={search_term}"
    print(f"Searching Imgur for '{search_term}' (using Selenium)...")

    driver = None

    try:
        driver = setup_selenium_in_colab()
        if driver is None:
            return

        driver.get(search_url)

        print("Waiting for page to load...")
        time.sleep(5)

        html = driver.page_source

    except Exception as e:
        print(f"Error: Selenium runtime failed. {e}")
        return
    finally:
        if driver:
            driver.quit()

    # web scraping, parse HTML with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # web scraping, find all image elements
    image_elements = soup.select('img[src*="i.imgur.com"]')

    if not image_elements:
        print("No images found. Imgur might have changed its layout, or the network timed out.")
        return

    print(f"Found {len(image_elements)} images (may include thumbnails). Starting download...")

    downloaded_count = 0
    downloaded_urls = set() # Use a set to prevent downloading duplicate URLs

    # loop through all found image and download
    for i, img in enumerate(image_elements):
        if downloaded_count >= max_images:
            print(f"Reached download limit of {max_images} images.")
            break

        image_url = img.get('src')

        if image_url.startswith('//'):
            image_url = 'https:' + image_url

        image_url = image_url.split('?')[0]

        if image_url.endswith('b.jpg'):
            image_url = image_url[:-5] + '.jpg'
        elif image_url.endswith('b.png'):
            image_url = image_url[:-5] + '.png'

        # prevent duplicate images
        if image_url in downloaded_urls:
            continue

        downloaded_urls.add(image_url)

        print(f"  Downloading image {downloaded_count + 1}: {image_url}")

        try:
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
            img_res = requests.get(image_url, headers=headers)
            img_res.raise_for_status()

            filename = os.path.basename(image_url)
            file_path = os.path.join(save_directory, filename)

            with open(file_path, 'wb') as image_file:
                for chunk in img_res.iter_content(100000):
                    image_file.write(chunk)

            downloaded_count += 1

        except requests.exceptions.RequestException as e:
            print(f"    -> Download failed (link may be dead): {e}")
        except IOError as e:
            print(f"    -> File save failed: {e}")

    print(f"\nDownload complete! Total of {downloaded_count} images saved to {save_directory} folder.")



In [3]:
##### ===== part 2 execution =====
if __name__ == "__main__":
    # get user information
    search_term = input("Please enter your image search term (e.g., cats): ")

    if not search_term:
        print("No search term entered. Exiting.")
        sys.exit()

    # get the number of images to download
    max_images_input = input(f"How many '{search_term}' images do you want to download? (Default: 10): ")

    try:
        max_images_to_download = int(max_images_input)
        if max_images_to_download <= 0:
            print("Input must be a positive number. Using default (10).")
            max_images_to_download = 10
    except ValueError:
        print("Invalid input. Using default (10).")
        max_images_to_download = 10

    print("-" * 20)
    download_imgur_images(search_term, max_images=max_images_to_download)

Please enter your image search term (e.g., cats): cat
How many 'cat' images do you want to download? (Default: 10): 10
--------------------
Images will be saved to: /content/imgur_cat
(You can find it in the 'Files' tab on the left sidebar)
Searching Imgur for 'cat' (using Selenium)...
Waiting for page to load...
Found 56 images (may include thumbnails). Starting download...
Reached download limit of 10 images.

Download complete! Total of 10 images saved to imgur_cat folder.
